In [107]:
import pandas as pd
import numpy as np
from statsmodels.stats import power

## Сокращение дисперсии

In [13]:
data = pd.read_csv("../data/marketing_AB.csv").drop(columns=['Unnamed: 0'])
data

,user id,test group,converted,total ads,most ads day,most ads hour
0,1069124,ad,False,130,Monday,20
1,1119715,ad,False,93,Tuesday,22
2,1144181,ad,False,21,Tuesday,18
3,1435133,ad,False,355,Tuesday,10
4,1015700,ad,False,276,Friday,14
...,...,...,...,...,...,...
588096,1278437,ad,False,1,Tuesday,23
588097,1327975,ad,False,1,Tuesday,23
588098,1038442,ad,False,3,Tuesday,23
588099,1496395,ad,False,1,Tuesday,23


In [10]:
data['test group'].value_counts()

test group
ad     564577
psa     23524
Name: count, dtype: int64

В этом эксперименте пользователи группы `ad` видели больше рекламы чем пользователи группы `psa` 

Сделаем две страшные вещи:
1. Будем сокращать дисперсию конверсии
2. Используем излишек группы `ad` в роли предэкспериментальных данных (в жизни собирайте эти данные честно)

In [ ]:
A = data[data['test group'] == 'psa'].reset_index(drop=True)
A.loc[:,'converted'] = A['converted'].astype(int)
B = data[data['test group'] == 'ad'].head(A.shape[0]).reset_index(drop=True)
B.loc[:,'converted'] = B['converted'].astype(int)
before_size = (data.shape[0] - 2 * A.shape[0]) // 2
A_before = data[data['test group'] == 'ad'].iloc[A.shape[0]:A.shape[0]+before_size, 2].astype(int).reset_index(drop=True)
B_before = data[data['test group'] == 'ad'].iloc[A.shape[0]+before_size:, 2].astype(int).reset_index(drop=True)

In [70]:
A.converted.var(), B.converted.var()

(0.01753608278252422, 0.04638327400686676)

Посмотрим, насколько можно сократить дисперсию с помощью стратификации и CUPED

### Стратификация

Для деления на страты используйте день недели с наибольшим количеством просмотров рекламы<br>
Найдите и сохраните дисперсию и среднее для `A.converted` и `B.converted`

In [71]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

A_strats = A.groupby(['most ads day']).converted.agg(['var', 'mean', ('weight', lambda x: len(x) / len(A))])
B_strats = B.groupby(['most ads day']).converted.agg(['var', 'mean', ('weight', lambda x: len(x) / len(B))])
A_strat_var = (A_strats['var'] * A_strats['weight']).sum()
B_strat_var = (B_strats['var'] * B_strats['weight']).sum()
A_strat_mean = (A_strats['mean'] * A_strats['weight']).sum()
B_strat_mean = (B_strats['mean'] * B_strats['weight']).sum()

A_strat_var, B_strat_var

(0.01753101785759261, 0.046056531146180704)

### CUPED

Найдите CUPED-значения, используя `A_before` и `B_before`

In [94]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
A_theta = A_before.cov(A.converted) / A_before.var()
B_theta = B_before.cov(B.converted) / B_before.var()
A_cuped = A.converted - ((A_before - A_before.mean()) * A_theta)[0]
B_cuped = B.converted - ((B_before - B_before.mean()) * B_theta)[0]

Снова дисперсию и среднее:

In [101]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
A_cuped.var(), B_cuped.var()

(0.017536082782524224, 0.04638327400686676)

Не надо использовать CUPED на таких данных?

### Сокращение дисперсии на знакомых задачах

Давайте вспомним задачу Cookie cats из первой темы, у её датасета тоже были большие дисперсии, мы даже убрали один выброс.

In [103]:
data = pd.read_csv("../data/cookie_cats.csv")
data = data[data.sum_gamerounds < data.sum_gamerounds.max()]
data

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True
...,...,...,...,...,...
90184,9999441,gate_40,97,True,False
90185,9999479,gate_40,30,False,False
90186,9999710,gate_30,28,True,False
90187,9999768,gate_40,51,True,False


In [104]:
data.groupby('version').sum_gamerounds.agg(['var', 'mean'])

,var,mean
version,,
gate_30,10415.753288,51.342111
gate_40,10669.736422,51.298776


Попробуйте поделить датасет на 4 страты по значениям retention, найдите стратифицированные среднее и дисперсию для группы A.

In [109]:
A = data[data.version == 'gate_30']

### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

A_strats = A.groupby(['retention_1', 'retention_7']).sum_gamerounds.agg(['var', 'mean', ('weight', lambda x: len(x) / len(A))])
A_strat_var = (A_strats['var'] * A_strats['weight']).sum()
A_strat_mean = (A_strats['mean'] * A_strats['weight']).sum()

A_strat_var

6974.336965830477

На основе группы A посчитайте количество экспериментов для t-теста в случае со стратификацией и без.

In [120]:
alpha = 0.05
beta = 0.8
lift = 0.1
ratio = 1

### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

effect_size = lift * A['sum_gamerounds'].mean() / A['sum_gamerounds'].std()
n = np.ceil(power.tt_ind_solve_power(effect_size=effect_size, ratio=ratio, power=beta, alpha=alpha))

cuped_effect_size = lift * A_strat_mean / np.sqrt(A_strat_var)
cuped_n = np.ceil(power.tt_ind_solve_power(effect_size=cuped_effect_size, ratio=ratio, power=beta, alpha=alpha))
n, cuped_n

(6204.0, 4155.0)